In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
from data import load_debt_data, total_annual_debt, total_annual_unemployment, filter_by_year, filter_by_years, filter_by_states, population_from_density, normalized_debt_per_capita, normalized_unemployment_per_capita, load_recipients_of_benefits, load_graduation_rates, get_dataset_unit, load_expenditure_on_public_schools


In [16]:
df = pd.read_csv('data/debt_92-05.csv', sep=';')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['time'] = pd.to_datetime(df['time']).dt.year

In [17]:
# get total annual debt by year for Berlin on a state level
filtered_df = df[df['1_variable_attribute_label'] == "Berlin"]
filtered_df = filtered_df[filtered_df['2_variable_attribute_label'] == "Länder"]
filtered_df['total_annual_debt'] = filtered_df.groupby('time')['value'].transform('sum')
filtered_df = filtered_df.groupby('time')['total_annual_debt'].agg(lambda x: list(x)[0]).reset_index()
filtered_df

,time,total_annual_debt
0,1992,13069.0
1,1993,16053.0
2,1994,18454.0
3,1995,23700.0
4,1996,26911.0
5,1997,29000.0
6,1998,31211.0
7,1999,33231.0
8,2000,34936.0
9,2001,39778.0


In [18]:
from data import load_population_density, population_from_density

df= load_population_density()

pop= population_from_density()

print(pop[pop['year']=='2015-12-31'])

          year  Baden-Württemberg    Bayern   Berlin  Brandenburg  Bremen  \
20  2015-12-31           10868608  12839918  3521616      2476068  645996   

    Hamburg   Hessen  Mecklenburg-Vorpommern  Niedersachsen  \
20  1786330  6186695                 1598937        7904588   

    Nordrhein-Westfalen  Rheinland-Pfalz  Saarland  Sachsen  Sachsen-Anhalt  \
20             17859492          4048788    996772  4069273         2248950   

    Schleswig-Holstein  Thüringen  
20             2853103    2167048  


In [19]:
from data import normalized_debt_per_capita, normalized_unemployment_per_capita


debt_norm = normalized_debt_per_capita()
unemp_norm = normalized_unemployment_per_capita()

print(debt_norm.head())
print(unemp_norm.head())


               state  year    value
0  Baden-Württemberg  1995  2481.92
1  Baden-Württemberg  1996  2632.99
2  Baden-Württemberg  1997  2727.84
3  Baden-Württemberg  1998  2831.82
4  Baden-Württemberg  1999  2893.37
               state  year  value
0  Baden-Württemberg  1995   3.18
1  Baden-Württemberg  1996   3.41
2  Baden-Württemberg  1997   3.67
3  Baden-Württemberg  1998   3.37
4  Baden-Württemberg  1999    3.1


In [20]:
from data import load_graduation_rates, load_recipients_of_benefits, normalized_unemployment_per_capita

unemployment = normalized_unemployment_per_capita()
graduation_rates = load_graduation_rates()
recipients = load_recipients_of_benefits()

print(unemployment.head())
print(graduation_rates.head())
print(recipients.head())

               state  year  value
0  Baden-Württemberg  1995   3.18
1  Baden-Württemberg  1996   3.41
2  Baden-Württemberg  1997   3.67
3  Baden-Württemberg  1998   3.37
4  Baden-Württemberg  1999    3.1
                state  year  value
16  Baden-Württemberg  2000   18.7
17             Bayern  2000   16.1
18             Berlin  2000   23.3
19        Brandenburg  2000    7.2
20             Bremen  2000   23.6
               state  year     value
0  Baden-Württemberg  2005  207015.0
1  Baden-Württemberg  2006  208433.0
2  Baden-Württemberg  2007  216440.0
3  Baden-Württemberg  2008  216079.0
4  Baden-Württemberg  2009  229110.0


In [21]:
#### Example of merging two variables

debt = load_debt_data()[['state', 'year', 'value']]
unemployment = total_annual_unemployment()

debt_grouped = debt.groupby(['state','year'], as_index=False).agg({'value': 'sum'})

# Debt data covers smaller span so using that now, extend to more features
min_year = min(debt['year'])
max_year = max(debt['year'])

unemployment = filter_by_years(unemployment, min_year, max_year)

combined = debt_grouped.sort_values('state')
combined['unemployment'] = unemployment.sort_values('state')['value'].values
combined = combined.rename(columns={'value': 'debt'})

combined

,state,year,debt,unemployment
0,Baden-Württemberg,1992,32024.0,191970
32,Baden-Württemberg,2024,463024.0,269990
31,Baden-Württemberg,2023,425074.0,245466
30,Baden-Württemberg,2022,434277.0,223119
29,Baden-Württemberg,2021,458651.0,247774
...,...,...,...,...
506,Thüringen,2003,16045.0,210664
507,Thüringen,2004,17021.0,207430
508,Thüringen,2005,17900.0,209941
501,Thüringen,1998,12403.0,209275


In [22]:
#### Testing encoding states as numbers

combined['state_enc']=combined['state'].astype('category').cat.codes
combined

,state,year,debt,unemployment,state_enc
0,Baden-Württemberg,1992,32024.0,191970,0
32,Baden-Württemberg,2024,463024.0,269990,0
31,Baden-Württemberg,2023,425074.0,245466,0
30,Baden-Württemberg,2022,434277.0,223119,0
29,Baden-Württemberg,2021,458651.0,247774,0
...,...,...,...,...,...
506,Thüringen,2003,16045.0,210664,15
507,Thüringen,2004,17021.0,207430,15
508,Thüringen,2005,17900.0,209941,15
501,Thüringen,1998,12403.0,209275,15


In [27]:
#### Testing extended combine_vars
def combine_features(feature_dict, chosen_features):
    '''
    Currently it is set up to make a new frame depending on chosen features since 
    not all features have the same years available.
    '''
    
    # Select relevant features
    # Always load debt
    debt = normalized_debt_per_capita()[['state', 'year', 'value']]
    debt_grouped = debt.groupby(['state','year'], as_index=False).agg({'value': 'sum'})
    combined = debt_grouped.sort_values(['state', 'year'])
    combined = combined.rename(columns={'value': 'Debt'})

    # Load other dataframes
    feature_frames = [feature_dict[feature] for feature in chosen_features]

    # Debt data covers smaller span so using that now, extend to more features
    min_year = max([min(df['year']) for df in feature_frames])
    max_year = min([max(df['year']) for df in feature_frames])

    # Filter all features to this interval
    combined = filter_by_years(combined, min_year, max_year)
    feature_frames = [filter_by_years(df, min_year, max_year) for df in feature_frames]

    # Put all value columns into combined frame
    for idx, feature in enumerate(chosen_features):
        combined[feature] = feature_frames[idx].sort_values(['state', 'year'])['value'].values

    return combined

In [24]:
features = {
    "Debt": normalized_debt_per_capita(),
    "Unemployment": normalized_unemployment_per_capita(),
    "Graduation Rates": load_graduation_rates(),
    "Recipients of Benefits": load_recipients_of_benefits(),
    "Expenditure on Public Schools": load_expenditure_on_public_schools()
}

chosen_features = ["Unemployment", "Graduation Rates", "Recipients of Benefits"]

In [28]:
combined = combine_features(features, chosen_features)
combined#[["state", "year", "Debt"]]

,state,year,Debt,Unemployment,Graduation Rates,Recipients of Benefits
10,Baden-Württemberg,2005,3827.57,3.59,21.2,207015.0
11,Baden-Württemberg,2006,3965.19,3.25,22.7,208433.0
12,Baden-Württemberg,2007,4007.3,2.52,25.0,216440.0
13,Baden-Württemberg,2008,4004.14,2.12,28.1,216079.0
14,Baden-Württemberg,2009,5365.96,2.64,33.9,229110.0
...,...,...,...,...,...,...
459,Thüringen,2019,27319.48,2.77,33.3,36509.0
460,Thüringen,2020,29549.24,3.15,30.2,10210.0
461,Thüringen,2021,31111.7,2.96,36.5,10340.0
462,Thüringen,2022,29597.85,2.75,35.8,10285.0
